In [1]:
from flask import Flask, request, render_template
import numpy as np
import pickle
from tensorflow.keras.models import load_model

# Load model and scalers
model = load_model('crop_model_dl.keras')
sc = pickle.load(open('standscaler.pkl', 'rb'))
ms = pickle.load(open('minmaxscaler.pkl', 'rb'))


/Users/vedniyasvyas/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:

# Flask App
app = Flask(__name__)

@app.route('/')
def index():
    return render_template("index.html")

@app.route('/recommend')
def recommend_page():
    return render_template("recommend.html")

@app.route("/crop-info/<crop_name>")
def crop_info(crop_name):
    crop_data = {
        "rice": {"temperature": "20°C - 35°C", "ph": "5.5 - 7", "rainfall": "1000mm - 2000mm", "soil": "Clayey or loamy soil", "image": "rice.jpg"},
        "maize": {"temperature": "18°C - 27°C", "ph": "5.5 - 7.5", "rainfall": "500mm - 800mm", "soil": "Fertile, well-drained loamy soil", "image": "maize.jpg"},
        "jute": {"temperature": "24°C - 37°C", "ph": "6 - 7.5", "rainfall": "1500mm - 2500mm", "soil": "Alluvial soil rich in loam", "image": "jute.jpg"},
        "cotton": {"temperature": "21°C - 30°C", "ph": "5.8 - 8", "rainfall": "600mm - 1200mm", "soil": "Black soil or alluvial soil", "image": "cotton.jpg"},
        "coconut": {"temperature": "21°C - 32°C", "ph": "5.2 - 8.6", "rainfall": "1000mm - 3000mm", "soil": "Sandy loam to alluvial soil", "image": "coconut.jpg"},
        "papaya": {"temperature": "25°C - 30°C", "ph": "5.5 - 7", "rainfall": "1000mm - 2000mm", "soil": "Well-drained loamy soil", "image": "papaya.jpg"},
        "orange": {"temperature": "10°C - 35°C", "ph": "5.5 - 7.5", "rainfall": "750mm - 2500mm", "soil": "Loamy soil rich in organic content", "image": "orange.jpg"},
        "apple": {"temperature": "6°C - 15°C", "ph": "6 - 7", "rainfall": "1000mm - 1250mm", "soil": "Loamy soil with good drainage", "image": "apple.jpg"},
        "muskmelon": {"temperature": "18°C - 30°C", "ph": "6 - 6.8", "rainfall": "400mm - 600mm", "soil": "Sandy loam to loamy soil", "image": "muskmelon.jpg"},
        "watermelon": {"temperature": "24°C - 30°C", "ph": "6 - 6.8", "rainfall": "500mm - 1000mm", "soil": "Loamy soil rich in organic matter", "image": "watermelon.jpg"},
        "grapes": {"temperature": "15°C - 40°C", "ph": "6.5 - 7.5", "rainfall": "700mm - 1200mm", "soil": "Well-drained sandy loam", "image": "grapes.jpg"},
        "mango": {"temperature": "24°C - 30°C", "ph": "5.5 - 7.5", "rainfall": "750mm - 2500mm", "soil": "Well-drained sandy loam", "image": "mango.jpg"},
        "banana": {"temperature": "26°C - 30°C", "ph": "5.5 - 7", "rainfall": "1000mm - 2000mm", "soil": "Well-drained fertile loamy soil", "image": "banana.jpg"},
        "pomegranate": {"temperature": "25°C - 35°C", "ph": "5.5 - 7.5", "rainfall": "500mm - 800mm", "soil": "Sandy loam to black soil", "image": "pomegranate.jpg"},
        "lentil": {"temperature": "18°C - 30°C", "ph": "6 - 7.5", "rainfall": "600mm - 1000mm", "soil": "Loamy to clay loam soil", "image": "lentil.jpg"},
        "blackgram": {"temperature": "25°C - 35°C", "ph": "6 - 7.5", "rainfall": "800mm - 1000mm", "soil": "Well-drained loamy soil", "image": "blackgram.jpg"},
        "mungbean": {"temperature": "25°C - 35°C", "ph": "6.2 - 7.2", "rainfall": "600mm - 1000mm", "soil": "Well-drained loamy to sandy loam", "image": "mungbean.jpg"},
        "mothbeans": {"temperature": "25°C - 35°C", "ph": "6.5 - 7.5", "rainfall": "500mm - 750mm", "soil": "Light textured soils", "image": "mothbeans.jpg"},
        "pigeonpeas": {"temperature": "26°C - 30°C", "ph": "5.5 - 7", "rainfall": "600mm - 1000mm", "soil": "Loamy to clay loam soils", "image": "pigeonpeas.jpg"},
        "kidneybeans": {"temperature": "18°C - 24°C", "ph": "6 - 7", "rainfall": "500mm - 800mm", "soil": "Fertile loamy soils", "image": "kidneybeans.jpg"},
        "chickpea": {"temperature": "20°C - 30°C", "ph": "6 - 9", "rainfall": "600mm - 1000mm", "soil": "Loamy to sandy soil", "image": "chickpea.jpg"},
        "coffee": {"temperature": "15°C - 28°C", "ph": "6 - 6.5", "rainfall": "1500mm - 2500mm", "soil": "Well-drained fertile soil", "image": "coffee.jpg"}
    }

    info = crop_data.get(crop_name.lower())
    if info:
        return render_template("crop_info.html", crop_name=crop_name.title(), info=info)
    elif crop_name  == 'all':
        for crop in crop_data:
            info = crop_data.get(crop.lower())
            if info:
                return render_template("crop_info.html", crop=crop.title(), info=info)
    else:
        return "Crop info not found!", 404

    
@app.route("/weather")
def weather():
    return render_template("weather.html")

@app.route("/predict", methods=['POST'])
def predict():
    try:
        N = float(request.form['Nitrogen'])
        P = float(request.form['Phosporus'])
        K = float(request.form['Potassium'])
        temp = float(request.form['Temperature'])
        humidity = float(request.form['Humidity'])
        ph = float(request.form['Ph'])
        rainfall = float(request.form['Rainfall'])
    except ValueError:
        return render_template('recommend.html', result="Invalid input. Please enter valid numerical values.")

    # Input processing
    features = np.array([[N, P, K, temp, humidity, ph, rainfall]])
    scaled = sc.transform(ms.transform(features))

    # Prediction
    prediction = model.predict(scaled)[0]
    top_indices = prediction.argsort()[-3:][::-1]

    crop_dict = {
        0: "Rice", 1: "Maize", 2: "Jute", 3: "Cotton", 4: "Coconut", 5: "Papaya",
        6: "Orange", 7: "Apple", 8: "Muskmelon", 9: "Watermelon", 10: "Grapes",
        11: "Mango", 12: "Banana", 13: "Pomegranate", 14: "Lentil", 15: "Blackgram",
        16: "Mungbean", 17: "Mothbeans", 18: "Pigeonpeas", 19: "Kidneybeans",
        20: "Chickpea", 21: "Coffee"
    }

    top_crops = [crop_dict.get(idx, "Unknown Crop") for idx in top_indices]
    result = "Top 3 recommended crops:\n"
    for i, crop in enumerate(top_crops, 1):
        result += f"{i}. {crop}\n"

    return render_template('recommend.html', result=result)


In [ ]:

if __name__ == "__main__":
    app.run(port=5001)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [19/Apr/2025 14:18:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2025 14:18:40] "GET /crop-info/all HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2025 14:18:40] "GET /static/images/rice.jpg HTTP/1.1" 304 -
127.0.0.1 - - [19/Apr/2025 14:18:57] "GET / HTTP/1.1" 200 -
